Homework Week 5 <br>
MSDS 600<br>
Bill Beemer<br>

Import Block

In [1]:
import pandas as pd
# from pycaret.classification import predict_model, load_model
from pycaret.classification import setup, compare_models, predict_model, save_model, load_model
# import matplotlib.pyplot as plt
# import math
# import seaborn as sns
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.tree import DecisionTreeClassifier, plot_tree
# from scikitplot.estimators import plot_feature_importances

%matplotlib inline


ImportError: cannot import name 'if_delegate_has_method' from 'sklearn.utils.metaestimators' (/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/utils/metaestimators.py)

Build dataframe from Prepped Churn Data

In [ ]:
df = pd.read_csv('data/new_churn_data.csv', index_col='customerID')
df

### ANALYSIS
<br>
After last weeks analysis I was certain that the monthly charge was the driving factor in Churn - but as this weeks lesson has shown - the MORE important feature is contract length - either month to month, one year or two year - the only options.  I am now curious about which contracts show the most Churn.  <br>
<br>
Above I was taking notes as I was going - so you may find some interesting analysis regarding why I droppped the TOTAL charges, since that feature seemed be to included in tenure and monthly charges and therefore was redundant.<br>
<br>
I also found that the default depth was 30 - wow! <br>
<br>
And finally, only removed the phoneservice feature (even though I also tested dropping PaymentMethod) - Once this was dropped, tenure and Monthly Payments swapped places behind contract type as important features. (and tenure displayed an even lower importance when PaymentMethod was also dropped, making the Contract length and MonthlyCharges appear to be the ONLY features playing a role.)<br>
